In [1]:
require 'hdf5'

{
  H5Z_FILTER_CONFIG_ENCODE_ENABLED : 1
  H5F_ACC_RDWR : 1
  _getTorchType : function: 0x070194e8
  H5F_OBJ_FILE : 1
  H5S_ALL : 0
  H5F_OBJ_GROUP : 4
  C : userdata: 0x073b3748
  H5P_DEFAULT : 0
  _describeObject : function: 0x07019568
  H5Z_FILTER_NBIT : 5
  _debugMode : false
  _getObjectType : function: 0x07019528
  H5F_OBJ_ALL : 31
  _getObjectName : function: 0x07019508
  version : 
    {
      1 : 1
      2 : 8
      3 : 15
    }
  H5Z_FILTER_SHUFFLE : 2
  HDF5Group : table: 0x07056180
  open : function: 0x0701fc98
  H5Z_FILTER_SZIP : 4
  H5F_OBJ_ATTR : 16
  H5Z_FILTER_FLETCHER32 : 3
  H5F_OBJ_DATATYPE : 8
  debugMode : function: 0x0701fc18
  H5F_ACC_EXCL : 4
  H5Z_FILTER_NONE : 0
  _testUtils : 
    {
      deepAlmostEq : function: 0x07010ac8
      withTmpDir : function: 0x07000410
    }
  _nativeTypeForTensorType : function: 0x073b3940
  H5F_ACC_TRUNC : 2
  _config : 
    {
      HDF5_INCLUDE_PATH : /Users/nicolasdrizard/anaconda/include
      HDF5_LIBRARIES : /Users/nicolasd

     NATIVE_INT : 50331660
      BITFIELD : 4
      NATIVE_UINT_LEAST16 : 50331673
      NATIVE_INT_LEAST16 : 50331672
      IEEE_F64LE : 50331704
      NATIVE_INT_FAST64 : 50331686
      NATIVE_UINT_FAST64 : 50331687
      NATIVE_UINT_LEAST64 : 50331685
      NATIVE_INT_LEAST64 : 50331684
      ENUM : 8
      NATIVE_UINT64 : 50331683
      NATIVE_HBOOL : 50331701
      NATIVE_ULONG : 50331663
      NATIVE_INT_FAST32 : 50331680
      NATIVE_HADDR : 50331697
      NATIVE_UINT : 50331661
      NCLASSES : 11
      NATIVE_UINT_LEAST32 : 50331679
      NATIVE_INT_LEAST32 : 50331678
      STD_U64LE : 50331722
      NATIVE_UINT32 : 50331677
      NATIVE_SHORT : 50331658
      NATIVE_INT32 : 50331676
      VLEN : 9
      ARRAY : 10
      STD_U16LE : 50331718
      STD_B16LE : 50331726
      STD_I64LE : 50331714
      NATIVE_UINT16 : 50331671
      NATIVE_UINT_FAST8 : 50331669
      STD_B16BE : 50331727
      NATIVE_INT_FAST8 : 50331668
      FLOAT : 1
      REFERENCE : 7
      STD_U32LE : 5033

In [2]:
myFile = hdf5.open('PTB.hdf5','r')
train_input_word_windows = myFile:read('train_input_word_windows'):all()
train_input_cap_windows = myFile:read('train_input_cap_windows'):all()
valid_input_word_windows = myFile:read('valid_input_word_windows'):all()
valid_input_cap_windows = myFile:read('valid_input_cap_windows'):all()
train_output = myFile:read('train_output'):all()
valid_output = myFile:read('valid_output'):all()
nclasses = myFile:read('nclasses'):all()[1]
nwords = myFile:read('nwords'):all()[1]
ncap = 4

In [3]:
function NaiveBayes(Xtrain_words, Xtrain_cap, Ytrain, nwords, ncap, nclasses, alpha)
    local n = Ytrain:size(1)
    local window_size = Xtrain_words:size(2)
    
    -- Building the prior
    local prior = torch.zeros(nclasses)
    for i = 1, n do
        c = Ytrain[i]
        prior[c] = prior[c] + 1
    end
    prior:div(n)

    -- Building the count matrix for words and caps with alpha as smoothing parameter
    
    local F_word = torch.zeros(nwords, nclasses)
    local F_cap = torch.zeros(ncap, nclasses)
    local x_word = torch.DoubleTensor(window_size)
    local x_cap = torch.DoubleTensor(window_size)
    -- Alpha smoothing parameter only use for the words features
    F_word:fill(alpha)
    for i = 1, n do
        c = Ytrain[i]
        x_word:copy(Xtrain_words[i])
        F_word:narrow(2,c,1):indexAdd(1, x_word:type('torch.LongTensor'), torch.ones(window_size,1))
        x_cap:copy(Xtrain_cap[i])
        F_cap:narrow(2,c,1):indexAdd(1, x_cap:type('torch.LongTensor'), torch.ones(window_size,1))
    end

    -- Building p(x|y) (same memory loc as F_word)
    local x_conditional_y_word = F_word
    -- Normalization
    x_conditional_y_word:cdiv(torch.expand(x_conditional_y_word:sum(1), nwords, nclasses))

    -- Building p(x|y) (same memory loc as F_cap)
    local x_conditional_y_cap = F_cap
    -- Normalization
    x_conditional_y_cap:cdiv(torch.expand(x_conditional_y_cap:sum(1), ncap, nclasses))
    
    return prior, x_conditional_y_word, x_conditional_y_cap
end

In [5]:
function predict_NB(Xword, Xcap, prior, x_conditional_y_word, x_conditional_y_cap, nclasses, Y)
    local n = Xword:size(1)
    local window_size = Xword:size(2)
    -- Building the conditional p(y|x)
    local y_conditional_x = torch.ones(n, nclasses)
    local x_word = torch.DoubleTensor(window_size)
    local x_cap = torch.DoubleTensor(window_size)
    for i = 1, n do
        x_word:copy(Xword[i])
        x_cap:copy(Xcap[i])
        for j=1, window_size do
            y_conditional_x[i]:cmul(x_conditional_y_word:narrow(1, x_word[j], 1))
            y_conditional_x[i]:cmul(x_conditional_y_cap:narrow(1, x_cap[j], 1))
        end
        -- Multiplying with the prior
        y_conditional_x[i]:cmul(prior)
    end
    -- Predicting
    max, Ypred = y_conditional_x:max(2)
    -- Computing accuracy if Y provided
    if Y then
        right = 0
        for i = 1, n do
            if Y[i] == Ypred[i][1] then
                right = right + 1
            end
        end
        accuracy = right / n
        print("Accuracy: " .. accuracy)
        return Ypred, accuracy
    else
        return Ypred
    end
end

In [32]:
-- Script to train and test the NB
alpha = 1.
prior, x_conditional_y_word, x_conditional_y_cap = NaiveBayes(train_input_word_windows, train_input_cap_windows, train_output, nwords, ncap, nclasses, alpha)
Ypred_train, accuracy_train = predict_NB(train_input_word_windows, train_input_cap_windows, prior, x_conditional_y_word, x_conditional_y_cap, nclasses, train_output)
Ypred_validate, accuracy_validate = predict_NB(valid_input_word_windows, valid_input_cap_windows, prior, x_conditional_y_word, x_conditional_y_cap, nclasses, valid_output)


Accuracy: 0.37055615088105	


Accuracy: 0.3549556931294	


In [12]:
-- We add to the features here the position of the type feature in the window
train_word = myFile:read('train_input_word_windows'):all()
train_cap = myFile:read('train_input_cap_windows'):all()
valid_word = myFile:read('valid_input_word_windows'):all()
valid_cap = myFile:read('valid_input_cap_windows'):all()
test_word = myFile:read('test_input_word_windows'):all()
test_cap = myFile:read('test_input_cap_windows'):all()

for j = 1, 5 do
    train_word:narrow(2,j,1):add((j-1)*100002)
    valid_word:narrow(2,j,1):add((j-1)*100002)
end
for j = 1, 5 do
    train_cap:narrow(2,j,1):add((j-1)*4)
    valid_cap:narrow(2,j,1):add((j-1)*4)
end
for j = 1, 5 do
    test_word:narrow(2,j,1):add((j-1)*4)
    test_cap:narrow(2,j,1):add((j-1)*4)
end

nwords_bis = nwords * 5
ncap_bis = 4 * 5

In [37]:
-- Script to train and test the NB
alpha = 1.

timer = torch.Timer()

prior, x_conditional_y_word, x_conditional_y_cap = NaiveBayes(train_word, train_cap, train_output, nwords_bis, ncap_bis, nclasses, alpha)
Ypred_train, accuracy_train = predict_NB(train_word, train_cap, prior, x_conditional_y_word, x_conditional_y_cap, nclasses, train_output)
Ypred_validate, accuracy_validate = predict_NB(valid_word, valid_cap, prior, x_conditional_y_word, x_conditional_y_cap, nclasses, valid_output)

print('Time elapsed for NB ' .. timer:time().real .. ' seconds',"\n")

Accuracy: 0.90317925725293	


Accuracy: 0.885302864773	
Time elapsed for NB 77.999620914459 seconds	
	


# Tuning the alpha

In [17]:
-- Parameter
alphas = torch.DoubleTensor({0, 0.5, 0.75, 1., 1.25, 1.5, 1.75, 2, 2.5, 3.})
num_parameters2 = alphas:size(1)

In [18]:
-- We add to the features here the position of the type feature in the window
train_word = myFile:read('train_input_word_windows'):all()
train_cap = myFile:read('train_input_cap_windows'):all()
valid_word = myFile:read('valid_input_word_windows'):all()
valid_cap = myFile:read('valid_input_cap_windows'):all()
test_word = myFile:read('test_input_word_windows'):all()
test_cap = myFile:read('test_input_cap_windows'):all()

for j = 1, 5 do
    train_word:narrow(2,j,1):add((j-1)*100002)
    valid_word:narrow(2,j,1):add((j-1)*100002)
end
for j = 1, 5 do
    train_cap:narrow(2,j,1):add((j-1)*4)
    valid_cap:narrow(2,j,1):add((j-1)*4)
end
for j = 1, 5 do
    test_word:narrow(2,j,1):add((j-1)*4)
    test_cap:narrow(2,j,1):add((j-1)*4)
end

nwords_bis = nwords * 5
ncap_bis = 4 * 5

In [19]:
-- Metric storage

-- Initialization
training_accuracy2 = torch.zeros(num_parameters2)
valid_accuracy2 = torch.zeros(num_parameters2)
training_time2 = torch.zeros(num_parameters2)

In [29]:
-- Script to tune the hyperparameter alpha
for i=1, num_parameters2 do
    alpha = alphas[i]
    timer = torch.Timer()

    prior, x_conditional_y_word, x_conditional_y_cap = NaiveBayes(train_word, train_cap, train_output, nwords_bis, ncap_bis, nclasses, alpha)
    training_time2[i] = timer:time().real
    timer2 = torch.Timer()
    print('Time elapsed for training NB ' .. training_time2[i] .. ' seconds',"\n")
    Ypred_train, training_accuracy2[i] = predict_NB(train_word, train_cap, prior, x_conditional_y_word, x_conditional_y_cap, nclasses, train_output)
    Ypred_validate, valid_accuracy2[i] = predict_NB(valid_word, valid_cap, prior, x_conditional_y_word, x_conditional_y_cap, nclasses, valid_output)
    print('Time elapsed for predicting NB ' .. training_time2[i] .. ' seconds',"\n")
    
end

Time elapsed for training NB 35.085270881653 seconds	
	


Accuracy: 0.97311502784151	


Accuracy: 0.73702658412236	
Time elapsed for predicting NB 35.085270881653 seconds	
	


Time elapsed for training NB 30.807815074921 seconds	
	


Accuracy: 0.92764932625955	


...

Accuracy: 0.85880376830078	


Accuracy: 0.84595775673707	
Time elapsed for predicting NB 27.608752012253 seconds	
	


Time elapsed for training NB 26.461882829666 seconds	
	

Accuracy: 0.84907293577278	


Accuracy: 0.83739985433358	
Time elapsed for predicting NB 26.461882829666 seconds	
	


In [34]:
-- Saving (rewriting the files with update at each iteration)
filename = '../data/nb_hyperparameter.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('valid_accuracy2', valid_accuracy2)
myFile:write('training_accuracy2', training_accuracy2)
myFile:write('training_time2', training_time2)
myFile:close()

In [37]:
valid_accuracy2

 0.7370
 0.9061
 0.8951
 0.8853
 0.8763
 0.8687
 0.8617
 0.8562
 0.8460
 0.8374
[torch.DoubleTensor of size 10]

